In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from functools import partial

In [3]:
mnist = input_data.read_data_sets("/tmp/data/")
means=mnist.train.images.mean(axis=0, keepdims=True)
stds=mnist.train.images.std(axis=0, keepdims=True)+1e-10

n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
tf.reset_default_graph()

batch_norm_momentum=0.9

X=tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y=tf.placeholder(tf.int64, shape=(None), name='y')
training=tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope('dnn'):
    he_init=tf.contrib.layers.variance_scaling_initializer()
    
    my_batch_norm_layer=partial(tf.layers.batch_normalization, training=training, momentum=batch_norm_momentum)
    my_dense_layer=partial(tf.layers.dense, kernel_initializer=he_init)
    
    hidden1=my_dense_layer(X, n_hidden1, name='hidden1')
    bn1=tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2=my_dense_layer(bn1, n_hidden2, name='hidden2')
    bn2=tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn=my_dense_layer(bn2, n_outputs, name='outputs')
    logits=my_batch_norm_layer(logits_before_bn)
    
with tf.name_scope('loss'):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss=tf.reduce_mean(xentropy, name='loss')

learning_rate=0.01
with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(logits, y, 1)
    accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()
loss_summary=tf.summary.scalar('loss', loss)
file_writer=tf.summary.FileWriter('./tf_logs/dnn2', tf.get_default_graph())
n_epochs=40
batch_size=50

In [10]:
import time
start=time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch=mnist.train.next_batch(batch_size)
            X_batch_scaled=(X_batch-means)/stds
            summary_str=loss_summary.eval(feed_dict={X:X_batch, y:y_batch})
            step=epoch*(mnist.train.num_examples//batch_size)+iteration
            
            file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        
        if epoch % 2 == 0:
            acc_train=accuracy.eval(feed_dict={X:X_batch, y:y_batch})
            X_val_scaled=(mnist.validation.images-means)/stds
            acc_test=accuracy.eval(feed_dict={X:X_val_scaled, y:mnist.validation.labels})
            print(epoch, "batch accuracy:", acc_train, 'validation acuracy:', acc_test)
    
    saver_path=saver.save(sess, './my_model_final_dnn.ckpt')
end=time.time()
print(end-start)

0 batch accuracy: 0.9 validation acuracy: 0.826
2 batch accuracy: 0.96 validation acuracy: 0.8374
4 batch accuracy: 0.96 validation acuracy: 0.8438
6 batch accuracy: 0.92 validation acuracy: 0.8442
8 batch accuracy: 0.96 validation acuracy: 0.843
10 batch accuracy: 0.9 validation acuracy: 0.8462
12 batch accuracy: 0.96 validation acuracy: 0.844
14 batch accuracy: 1.0 validation acuracy: 0.838
16 batch accuracy: 0.98 validation acuracy: 0.8334
18 batch accuracy: 0.96 validation acuracy: 0.8292
20 batch accuracy: 0.98 validation acuracy: 0.8226
22 batch accuracy: 1.0 validation acuracy: 0.8244
24 batch accuracy: 0.98 validation acuracy: 0.8218
26 batch accuracy: 1.0 validation acuracy: 0.8134
28 batch accuracy: 1.0 validation acuracy: 0.8042
30 batch accuracy: 1.0 validation acuracy: 0.796
32 batch accuracy: 1.0 validation acuracy: 0.794
34 batch accuracy: 1.0 validation acuracy: 0.7922
36 batch accuracy: 1.0 validation acuracy: 0.7848
38 batch accuracy: 1.0 validation acuracy: 0.7808
27